In [3]:
import numpy as np
import tensorflow as tf

In [ ]:
# Dataset's requires the following:

# 1. Importing Data. Create a Dataset instance from some data
# 2. Create an Iterator. By using the created dataset to make an Iterator instance to 
#    iterate thought the dataset
# 3. Consuming Data. By using the created iterator we can get the elements from the 
#    dataset to feed the model

In [ ]:
# 1. Creating Dataset instance

# using one or more numpy arrays (most general case)

dataset = tf.data.Dataset.from_tensor_slices(x)

features, labels = (np.random.sample((100,2)), np.random.sample((100,1)))
dataset = tf.data.Dataset.from_tensor_slices((features,labels))

# using a tensor
dataset = tf.data.Dataset.from_tensor_slices(tf.random_uniform([100, 2]))

# using a placeholder
x = tf.placeholder(tf.float32, shape=[None,2])
dataset = tf.data.Dataset.from_tensor_slices(x)

# using a generator
sequence = np.array([[1],[2,3],[3,4]])
def generator():
    for el in sequence:
        yield el
dataset = tf.data.Dataset().from_generator(generator,
                                           output_types=tf.float32, 
                                           output_shapes=[tf.float32])

# 2. Creating an iterator


# 2.1 one_shot_iterator()
x = np.random.sample((100,2))
dataset = tf.data.Dataset.from_tensor_slices(x)
iter = dataset.make_one_shot_iterator()
el = iter.get_next()

with tf.Session() as sess:
    print(sess.run(el))

# 2.2 Initializable iterator

x = tf.placeholder(tf.float32, shape=[None,2])
dataset = tf.data.Dataset.from_tensor_slices(x)
data = np.random.sample((100,2))
iter = dataset.make_initializable_iterator() # create the iterator
el = iter.get_next()
with tf.Session() as sess:
    # feed the placeholder with data
    sess.run(iter.initializer, feed_dict={ x: data }) 
    print(sess.run(el))

# 2.3 Reinitializable iterator
# same as Initializable iterator but instead one, we create two datasets
# create two datasets, one for training and one for test
train_dataset = tf.data.Dataset.from_tensor_slices(train_data)
test_dataset = tf.data.Dataset.from_tensor_slices(test_data)

# create a iterator of the correct shape and type
iter = tf.data.Iterator.from_structure(train_dataset.output_types,
                                           train_dataset.output_shapes)
# create two initialisation operations
train_init_op = iter.make_initializer(train_dataset)
test_init_op = iter.make_initializer(test_dataset)
features, labels = iter.get_next()

# 2.4 Feedable iterator
# Basically instead of switch between datasets they switch between iterators 
# so you can have, for example, one iterator from make_one_shot_iterator() 
# and one from ` make_initializable_iterator().

# 3. Consuming data
next_el = iter.get_next()
print(sess.run(next_el)) # will output the current element

# In order to pass the data to a model we have to just pass the 
# tensors generated from get_next()

In [ ]:
# Other Stuff
# for batching the dataset
dataset = dataset.batch(BATCH_SIZE)
# number of times the dataset to be iterated
dataset = dataset.repeat()
# shuffles the dataset by default every epoch.
dataset = dataset.shuffle(buffer_size=100)
# custom function to each member of a dataset using the map method.
dataset = dataset.map(lambda x: x*2)

In [4]:
x = np.random.sample((10,2))
print(x)

[[ 0.00554269  0.51342026]
 [ 0.14084252  0.70982468]
 [ 0.31480329  0.29921774]
 [ 0.06010402  0.44220846]
 [ 0.30972163  0.74748617]
 [ 0.39255886  0.81211734]
 [ 0.69527971  0.84765305]
 [ 0.95113042  0.885299  ]
 [ 0.89517676  0.68355682]
 [ 0.75130352  0.49364485]]


In [6]:
# make a dataset from a numpy array
dataset = tf.data.Dataset.from_tensor_slices(x)

iter = dataset.make_one_shot_iterator()
el = iter.get_next()

with tf.Session() as sess:
    print(sess.run(el))
    print(sess.run(el))

[ 0.00554269  0.51342026]
[ 0.14084252  0.70982468]


In [7]:
# using two numpy arrays
features, labels = (np.random.sample((100,2)), np.random.sample((100,1)))
dataset = tf.data.Dataset.from_tensor_slices((features,labels))

iter = dataset.make_one_shot_iterator()
el = iter.get_next()

with tf.Session() as sess:
    print(sess.run(el))

(array([ 0.41384178,  0.46332406]), array([ 0.41036219]))


In [8]:
# using a tensor
dataset = tf.data.Dataset.from_tensor_slices(tf.random_uniform([100, 2]))

iter = dataset.make_initializable_iterator()
el = iter.get_next()

with tf.Session() as sess:
    sess.run(iter.initializer)
    print(sess.run(el))

[ 0.92488623  0.87302363]


In [9]:
# using a placeholder
x = tf.placeholder(tf.float32, shape=[None,2])
dataset = tf.data.Dataset.from_tensor_slices(x)

data = np.random.sample((100,2))

iter = dataset.make_initializable_iterator()
el = iter.get_next()

with tf.Session() as sess:
    sess.run(iter.initializer, feed_dict={ x: data })
    print(sess.run(el))

[ 0.25961897  0.56740773]


In [10]:
# from generator
sequence = np.array([[1],[2,3],[3,4]])

def generator():
    for el in sequence:
        yield el

dataset = tf.data.Dataset().from_generator(generator,
                                           output_types=tf.float32, 
                                           output_shapes=[tf.float32])
iter = dataset.make_initializable_iterator()
el = iter.get_next()

with tf.Session() as sess:
    sess.run(iter.initializer)
    print(sess.run(el))

[ 1.]


In [11]:
# initializable iterator to switch between datasets (train/test)
EPOCHS = 10

x, y = tf.placeholder(tf.float32, shape=[None,2]), tf.placeholder(tf.float32, shape=[None,1])
dataset = tf.data.Dataset.from_tensor_slices((x, y))

train_data = (np.random.sample((100,2)), np.random.sample((100,1)))
test_data = (np.array([[1,2]]), np.array([[0]]))

iter = dataset.make_initializable_iterator()
features, labels = iter.get_next()

with tf.Session() as sess:
#     initialise iterator with train data
    sess.run(iter.initializer, feed_dict={ x: train_data[0], y: train_data[1]})
    for _ in range(EPOCHS):
        sess.run([features, labels])
#     switch to test data
    sess.run(iter.initializer, feed_dict={ x: test_data[0], y: test_data[1]})
    print(sess.run([features, labels]))

[array([ 1.,  2.], dtype=float32), array([ 0.], dtype=float32)]


In [12]:
# Reinitializable iterator to switch between Datasets
EPOCHS = 10
# making fake data using numpy
train_data = (np.random.sample((100,2)), np.random.sample((100,1)))
test_data = (np.random.sample((10,2)), np.random.sample((10,1)))
# create two datasets, one for training and one for test
train_dataset = tf.data.Dataset.from_tensor_slices(train_data)
test_dataset = tf.data.Dataset.from_tensor_slices(test_data)
# create a iterator of the correct shape and type
iter = tf.data.Iterator.from_structure(train_dataset.output_types,
                                           train_dataset.output_shapes)
features, labels = iter.get_next()
# create the initialisation operations
train_init_op = iter.make_initializer(train_dataset)
test_init_op = iter.make_initializer(test_dataset)

with tf.Session() as sess:
    sess.run(train_init_op) # switch to train dataset
    for _ in range(EPOCHS):
        sess.run([features, labels])
    sess.run(test_init_op) # switch to val dataset
    print(sess.run([features, labels]))

[array([ 0.94524631,  0.05221903]), array([ 0.29584404])]


In [13]:
# BATCHING
BATCH_SIZE = 4
x = np.random.sample((100,2))
# make a dataset from a numpy array
dataset = tf.data.Dataset.from_tensor_slices(x).batch(BATCH_SIZE)

iter = dataset.make_one_shot_iterator()
el = iter.get_next()

with tf.Session() as sess:
    print(sess.run(el))

[[ 0.01522746  0.84285457]
 [ 0.5545286   0.43503361]
 [ 0.59391819  0.79048957]
 [ 0.19106855  0.54254723]]


In [14]:
# REPEAT
BATCH_SIZE = 4
x = np.array([[1],[2],[3],[4]])
# make a dataset from a numpy array
dataset = tf.data.Dataset.from_tensor_slices(x)
dataset = dataset.repeat()

iter = dataset.make_one_shot_iterator()
el = iter.get_next()

with tf.Session() as sess:
#     this will run forever
    while True:
        print(sess.run(el))

[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]


KeyboardInterrupt: 

In [15]:
# MAP
x = np.array([[1],[2],[3],[4]])
# make a dataset from a numpy array
dataset = tf.data.Dataset.from_tensor_slices(x)
dataset = dataset.map(lambda x: x*2)

iter = dataset.make_one_shot_iterator()
el = iter.get_next()

with tf.Session() as sess:
#     this will run forever
        for _ in range(len(x)):
            print(sess.run(el))

[2]
[4]
[6]
[8]


In [16]:
# SHUFFLE
BATCH_SIZE = 4
x = np.array([[1],[2],[3],[4]])
# make a dataset from a numpy array
dataset = tf.data.Dataset.from_tensor_slices(x)
dataset = dataset.shuffle(buffer_size=100)
dataset = dataset.batch(BATCH_SIZE)

iter = dataset.make_one_shot_iterator()
el = iter.get_next()

with tf.Session() as sess:
    print(sess.run(el))

[[4]
 [3]
 [1]
 [2]]


In [17]:
# how to pass the value to a model
EPOCHS = 10
BATCH_SIZE = 16
# using two numpy arrays
features, labels = (np.array([np.random.sample((100,2))]), 
                    np.array([np.random.sample((100,1))]))

dataset = tf.data.Dataset.from_tensor_slices((features,labels)).repeat().batch(BATCH_SIZE)

iter = dataset.make_one_shot_iterator()
x, y = iter.get_next()

# make a simple model
net = tf.layers.dense(x, 8, activation=tf.tanh) # pass the first value from iter.get_next() as input
net = tf.layers.dense(net, 8, activation=tf.tanh)
prediction = tf.layers.dense(net, 1, activation=tf.tanh)

loss = tf.losses.mean_squared_error(prediction, y) # pass the second value from iter.get_net() as label
train_op = tf.train.AdamOptimizer().minimize(loss)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(EPOCHS):
        _, loss_value = sess.run([train_op, loss])
        print("Iter: {}, Loss: {:.4f}".format(i, loss_value))

Iter: 0, Loss: 0.6187
Iter: 1, Loss: 0.6027
Iter: 2, Loss: 0.5868
Iter: 3, Loss: 0.5712
Iter: 4, Loss: 0.5558
Iter: 5, Loss: 0.5406
Iter: 6, Loss: 0.5256
Iter: 7, Loss: 0.5109
Iter: 8, Loss: 0.4964
Iter: 9, Loss: 0.4822


In [18]:
# Wrapping all together -> Switch between train and test set
EPOCHS = 10
BATCH_SIZE = 16
# create a placeholder to dynamically switch between batch sizes
batch_size = tf.placeholder(tf.int64)

x, y = tf.placeholder(tf.float32, shape=[None,2]), tf.placeholder(tf.float32, shape=[None,1])
dataset = tf.data.Dataset.from_tensor_slices((x, y)).batch(batch_size).repeat()

# using two numpy arrays
train_data = (np.random.sample((100,2)), np.random.sample((100,1)))
test_data = (np.random.sample((20,2)), np.random.sample((20,1)))

n_batches = len(train_data[0]) // BATCH_SIZE

iter = dataset.make_initializable_iterator()
features, labels = iter.get_next()
# make a simple model
net = tf.layers.dense(features, 8, activation=tf.tanh) # pass the first value from iter.get_next() as input
net = tf.layers.dense(net, 8, activation=tf.tanh)
prediction = tf.layers.dense(net, 1, activation=tf.tanh)

loss = tf.losses.mean_squared_error(prediction, labels) # pass the second value from iter.get_net() as label
train_op = tf.train.AdamOptimizer().minimize(loss)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    # initialise iterator with train data
    sess.run(iter.initializer, feed_dict={ x: train_data[0], y: train_data[1], batch_size: BATCH_SIZE})
    print('Training...')
    for i in range(EPOCHS):
        tot_loss = 0
        for _ in range(n_batches):
            _, loss_value = sess.run([train_op, loss])
            tot_loss += loss_value
        print("Iter: {}, Loss: {:.4f}".format(i, tot_loss / n_batches))
    # initialise iterator with test data
    sess.run(iter.initializer, feed_dict={ x: test_data[0], y: test_data[1], batch_size: test_data[0].shape[0]})
    print('Test Loss: {:4f}'.format(sess.run(loss)))

Training...
Iter: 0, Loss: 0.1269
Iter: 1, Loss: 0.1078
Iter: 2, Loss: 0.0922
Iter: 3, Loss: 0.0953
Iter: 4, Loss: 0.0864
Iter: 5, Loss: 0.0805
Iter: 6, Loss: 0.0873
Iter: 7, Loss: 0.0911
Iter: 8, Loss: 0.0899
Iter: 9, Loss: 0.0822
Test Loss: 0.078720
